# AllComponents Test Run

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AseiSugiyama/TFXTestRun/blob/master/notebooks/AllComponentsTestRun.ipynb)

## Set up

TFX requires apache-airflow and docker SDK.

In [ ]:
!pip install 'apache-airflow[gcp_api]' docker tfx

[TFMA (TensorFlow Model Analysis)](https://github.com/tensorflow/model-analysis) requires some jupyter extensions.

In [ ]:
# If you run this notebook on python3 runtime of Google Colab, you should install TFMA in python 2 environment.
# See also : https://github.com/googlecolab/colabtools/issues/60
# !pip2 install tensorflow_model_analysis -U
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension install --py --symlink tensorflow_model_analysis
!jupyter nbextension enable --py tensorflow_model_analysis

In this notebook, we use TFX version 0.13.0

In [ ]:
import tfx
tfx.version.__version__

TFX requires TensorFlow >= 1.13.1

In [ ]:
import tensorflow as tf
tf.__version__

TFX supports Python 3.5 from version 0.13.0

In [ ]:
import sys
sys.version

## Download sample data

In [ ]:
%%bash
# This enables you to run this notebook twice.
# There should not be train/eval files at ~/taxi/data, since TFX can handle only single file with version 0.13.0
if [ -e ~/taxi/data ]; then
    rm -rf ~/taxi/data
fi

# download taxi data
mkdir -p ~/taxi/data/simple
mkdir -p ~/taxi/serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O ~/taxi/data/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/r0.13/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O ~/taxi/taxi_utils.py

## Import

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from google.protobuf import json_format

from tfx.components.base.base_component import ComponentOutputs
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.orchestration.pipeline import Pipeline
from tfx.orchestration.tfx_runner import TfxRunner
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input
from tfx.utils.channel import Channel
from tfx.utils import types

## configs

In [ ]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')

# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

## Create ExampleGen

In [ ]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
train_config = example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
eval_config = example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        train_config,
        eval_config
    ]))

# Create outputs
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root, 'csv_example_gen/train/')

eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root, 'csv_example_gen/eval/')

example_outputs = ComponentOutputs({
    'examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples, eval_examples]
    ),
    'training_examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples]
    ),
    'eval_examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[eval_examples]
    ),    
})

example_gen = CsvExampleGen(
    name="CSV ExampleGen Component",
    input_base=examples, # A Channel of 'ExternalPath' type, it contains path of data source.
    output_config=output_config,  # An example_gen_pb2.Output instance, it contains train-eval split ratio.
    outputs=example_outputs # dict from name to output channel, it will be stored example_gen.outputs
)

## Create StatisticsGen

In [ ]:
# Create outputs
train_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='train')
train_statistics.uri = os.path.join(_data_root, 'statistics_gen/train/')

eval_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='eval')
eval_statistics.uri = os.path.join(_data_root, 'statistics_gen/eval/')

statistics_outputs = ComponentOutputs({
    'output': Channel(
        type_name='ExampleStatisticsPath',
        static_artifact_collection=[train_statistics, eval_statistics]
    )
})

statistics_gen = StatisticsGen(
    name='StatisticsGen Component', # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    input_data=example_gen.outputs.examples, # A Channel of 'ExamplesPath' type, it is equal to example_outputs
    outputs=statistics_outputs # dict from name to output channel, it will be stored statistics_gen.outputs
)

## Create SchemaGen

In [ ]:
# Create outputs
train_schema_path = types.TfxType(type_name='SchemaPath', split='train')
train_schema_path.uri = os.path.join(_data_root, 'schema_gen/')

# NOTE: SchemaGen.executor can handle JUST ONE SchemaPath.
# Two or more SchemaPaths will cause ValueError
# such as "ValueError: expected list length of one but got 2".
schema_outputs = ComponentOutputs({
    'output':Channel(
        type_name='SchemaPath',
        static_artifact_collection=[train_schema_path] 
    )
})

infer_schema = SchemaGen(
    name='SchemaGen Component',  # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    stats=statistics_gen.outputs.output, # A Channel of 'ExampleStatisticsPath' type, it is equal to statistics_outputs
    outputs=schema_outputs # dict from name to output channel, it will be stored schema_gen.outputs
)

## Create ExampleValidator

In [ ]:
eval_validator_path = types.TfxType('ExampleValidationPath', split='eval')
eval_validator_path.uri = os.path.join(_data_root, 'eval/validation/')

# NOTE: ExampleValidator.executor can handle JUST ONE ExampleValidationPath
# since ExampleValidator should check only 'eval' data.
# Two or more ExampleValidationPath will cause ValueError
# such as "ValueError: expected list length of one but got 2".
validator_outputs = ComponentOutputs({
    'output':Channel(
        type_name='ExampleValidationPath',
        static_artifact_collection=[eval_validator_path]
    )
})

example_validator = ExampleValidator(
    stats=statistics_gen.outputs.output,
    schema=infer_schema.outputs.output,
    name='Example Validator',
    outputs=validator_outputs
)

## Create Transform

In [ ]:
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root,
                                  'transform/transformed_examples/train/')
eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root,
                                 'transform/transformed_examples/eval/')
transform_output = types.TfxType(type_name='TransformPath')
transform_output.uri = os.path.join(_data_root,
                                    'transform/transform_output/')

transform_outputs = ComponentOutputs({
    # Output of 'tf.Transform', which includes an exported 
    # Tensorflow graph suitable for both training and serving
    'transform_output':Channel(
        type_name='TransformPath',
        static_artifact_collection=[transform_output]
    ),
    # transformed_examples: Materialized transformed examples, which includes 
    # both 'train' and 'eval' splits.
    'transformed_examples':Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples, eval_examples]
    )
})

transform = Transform(
    name="Transform Component",
    input_data=example_gen.outputs.examples,
    schema=infer_schema.outputs.output,
    module_file=_taxi_module_file,
    outputs=transform_outputs
)

## Create Trainer

In [ ]:
model_exports = types.TfxType(type_name='ModelExportPath')
model_exports.uri = os.path.join(_data_root, 'trainer/current/')

trainer_outputs = ComponentOutputs({
    'output':Channel(
        type_name='ModelExportPath',
        static_artifact_collection=[model_exports]
    )
})

trainer = Trainer(
    name='Trainer Component',
    module_file=_taxi_module_file,
    transformed_examples=transform.outputs.transformed_examples,
    schema=infer_schema.outputs.output,
    transform_output=transform.outputs.transform_output,
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000),
    outputs=trainer_outputs
)


## Create Model Analyzer

In [ ]:
eval_output = types.TfxType('ModelEvalPath')
eval_output.uri = os.path.join(_data_root, 'eval_output/')

model_analyzer_outputs = ComponentOutputs({
    'output':
    Channel(
        type_name='ModelEvalPath',
        static_artifact_collection=[eval_output]),
})

feature_slicing_spec = evaluator_pb2.FeatureSlicingSpec(specs=[
    evaluator_pb2.SingleSlicingSpec(
        column_for_slicing=['trip_start_hour'])
])

model_analyzer = Evaluator(
    name='Evaluator Component',
    examples=example_gen.outputs.examples,
    model_exports=trainer.outputs.output,
    feature_slicing_spec=feature_slicing_spec,
    outputs=model_analyzer_outputs
)

In [ ]:
def _Do(self, input_dict, output_dict, exec_properties):
    import apache_beam as beam
    import tensorflow as tf
    import tensorflow_model_analysis as tfma
    from typing import Any, Dict, List, Text
    from tfx.components.base import base_executor
    from tfx.proto import evaluator_pb2
    from tfx.utils import io_utils
    from tfx.utils import path_utils
    from tfx.utils import types
    from google.protobuf import json_format

    """Runs a batch job to evaluate the eval_model against the given input.
    Args:
      input_dict: Input dict from input key to a list of Artifacts.
        - model_exports: exported model.
        - examples: examples for eval the model.
      output_dict: Output dict from output key to a list of Artifacts.
        - output: model evaluation results.
      exec_properties: A dict of execution properties.
        - feature_slicing_spec: JSON string of evaluator_pb2.FeatureSlicingSpec
          instance, providing the way to slice the data.
    Returns:
      None
    """
    if 'model_exports' not in input_dict:
      raise ValueError('\'model_exports\' is missing in input dict.')
    if 'examples' not in input_dict:
      raise ValueError('\'examples\' is missing in input dict.')
    if 'output' not in output_dict:
      raise ValueError('\'output\' is missing in output dict.')

    self._log_startup(input_dict, output_dict, exec_properties)

    # Extract input artifacts
    model_exports_uri = types.get_single_uri(input_dict['model_exports'])

    feature_slicing_spec = evaluator_pb2.FeatureSlicingSpec()
    json_format.Parse(exec_properties['feature_slicing_spec'],
                      feature_slicing_spec)
    slice_spec = self._get_slice_spec_from_feature_slicing_spec(
        feature_slicing_spec)

    output_uri = types.get_single_uri(output_dict['output'])

    eval_model_path = path_utils.eval_model_path(model_exports_uri)

    tf.logging.info('Using {} for model eval.'.format(eval_model_path))
    eval_shared_model = tfma.default_eval_shared_model(
        add_metrics_callbacks=[
                        # calibration_plot_and_prediction_histogram computes calibration plot and prediction
                        # distribution at different thresholds.
                        tfma.post_export_metrics.calibration_plot_and_prediction_histogram(),
                        # auc_plots enables precision-recall curve and ROC visualization at different thresholds.
                        tfma.post_export_metrics.auc_plots()
                    ],
        eval_saved_model_path=eval_model_path)

    tf.logging.info('Evaluating model.')
    with beam.Pipeline(argv=self._get_beam_pipeline_args()) as pipeline:
      # pylint: disable=expression-not-assigned
      (pipeline
       | 'ReadData' >> beam.io.ReadFromTFRecord(
           file_pattern=io_utils.all_files_pattern(
               types.get_split_uri(input_dict['examples'], 'eval')))
       |
       'ExtractEvaluateAndWriteResults' >> tfma.ExtractEvaluateAndWriteResults(
           eval_shared_model=eval_shared_model,
           slice_spec=slice_spec,
           output_path=output_uri))
    tf.logging.info(
        'Evaluation complete. Results written to {}.'.format(output_uri))

model_analyzer.executor.Do = _Do

## Create Model Validator

In [ ]:
blessing = types.TfxType(type_name='ModelBlessingPath')
blessing.uri = os.path.join(_data_root, 'model_validator/blessed/')

results = types.TfxType(type_name='ModelValidationPath')
results.uri = os.path.join(_data_root, 'model_validator/results/')

model_validator_outputs = ComponentOutputs({
    'blessing':
    Channel(
        type_name='ModelBlessingPath',
        static_artifact_collection=[blessing]),
    'results':
    Channel(
        type_name='ModelValidationPath',
        static_artifact_collection=[results]),
})

model_validator = ModelValidator(
    name='Model Validator Component',
    examples=example_gen.outputs.examples, 
    model=trainer.outputs.output,
    outputs=model_validator_outputs
)

## Create Pusher

In [ ]:
custom_config={
    # If custom_config contains 'cmle_serving_args', Pusher will try to push ml model to AI Platform (GCE).
    # However, this config will be deplecated in next release of TFX.
    # To run it localy, we use empty dictionary .
}

pusher = Pusher(
    name='Pusher Component',
    model_export=trainer.outputs.output,
    model_blessing=model_validator.outputs.blessing,
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)),
    outputs='',
    custom_config=custom_config,
)

## Create Pipeline

In [ ]:
pipeline = Pipeline(
    pipeline_name="TFX Pipeline",
    pipeline_root=_pipeline_root,
    components=[
        example_gen, 
        statistics_gen, 
        infer_schema, 
        example_validator,
        transform, 
        trainer, 
        model_analyzer, 
        model_validator,
        pusher,
    ]
)

## Execute

In [ ]:
_directrun_config = {
    'metadata_db_root':None,
    'metadata_connection_config':None,
    'additional_pipeline_args':{
        'beam_pipeline_args':None,
    },
    'docker_operator_cfg':None,
    'enable_cache':None,
}

class DirectRunner(TfxRunner):
    """Tfx runner on local"""
    
    def __init__(self, config=None):
        self._config = config or {}
    
    def run(self, pipeline):
        # Merge airflow-specific configs with pipeline args
        self._config.update(pipeline.pipeline_args)
        
        for component in pipeline.components:
            self._execute_component(component)
            
        return pipeline
            
    def _execute_component(self, component):
        # parse parameters
        input_dict = {key:value.get() for key, value in component.input_dict.items()}
        output_dict = {key: value.get() for key, value in component.outputs.get_all().items()}
        exec_properties = component.exec_properties
        
        # create executor
        additional_pipeline_args = self._config.get('additional_pipeline_args') or {}
        executor = component.executor(beam_pipeline_args=additional_pipeline_args.get('beam_pipeline_args'))
        
        executor.Do(input_dict, output_dict, exec_properties)

In [ ]:
pipeline = DirectRunner().run(pipeline)

## Check Result

We will check followings:

0. Artifacts
1. Statistics
2. Schema
3. Anomalies
4. Model Evaluation

### Check artifacts

In [ ]:
!ls -Rlhs /root/taxi/data/simple/

### Check statistics

`anomalies.pbtxt` is only simple text file. To visualize it, we are going to;

1. get the path of `anomalies.pbtxt`from `example_validator`
2. parse `anomalies.pbtxt` into anomalies (protobuf)
3. visualize schema with [tfdv](https://www.tensorflow.org/tfx/data_validation/get_started)

In [ ]:
def get_stats_directories(statistics_gen):
    output_dict = {key: value.get() for key, value in statistics_gen.outputs.get_all().items()}
    input_dict = {key:value.get() for key, value in statistics_gen.input_dict.items()}
    split_to_instance = {x.split: x for x in input_dict['input_data']}
    directories = [types.get_split_uri(output_dict['output'], split) for split, instance in split_to_instance.items()]
    return directories

directories = get_stats_directories(statistics_gen)

In [ ]:
import tensorflow_data_validation as tfdv
from tfx.utils import io_utils

train_stats = tfdv.load_statistics(io_utils.get_only_uri_in_dir(directories[0]))

In [ ]:
tfdv.visualize_statistics(train_stats)

In [ ]:
schema = tfdv.infer_schema(train_stats)
schema

### Check Schema

`schema.pbtxt` is only simple text file, we are going to;

1. get the path of `schema.pbtxt`from `schema_gen`
2. parse `schema.pbtxt` into schema (protobuf)
3. visualize schema with [tfdv](https://www.tensorflow.org/tfx/data_validation/get_started)

In [ ]:
# 1. get the path of `schema.pbtxt`
def get_schema_directory(infer_schema):
    output_dict = {key: value.get() for key, value in infer_schema.outputs.get_all().items()}
    input_dict = {key:value.get() for key, value in infer_schema.input_dict.items()}
    split_to_instance = {x.split: x for x in input_dict['stats']}
    directory = types.get_split_uri(output_dict['output'], 'train')
    return directory

schema_directory = get_schema_directory(infer_schema)

In [ ]:
# 2. parse schema.pbtxt

from pathlib import Path
from google.protobuf.text_format import Parse
from google.protobuf.message import Message
from tensorflow_metadata.proto.v0 import schema_pb2

def parse_schema_proto_string(schema_directory):
    path = Path(schema_directory)
    for filepath in path.glob('*'):
        with open(str(filepath), 'r') as file: # since we are using python 3.5, not 3.6+
            schema_proto_string = file.read()
            schema = schema_pb2.Schema()
            Parse(schema_proto_string, schema)
            return schema

schema = parse_schema_proto_string(schema_directory)

In [ ]:
# 3. visualize schema with tfdv

import tensorflow_data_validation as tfdv
tfdv.display_schema(schema)

### Check anomalies

`anomalies.pbtxt` is only simple text file. To visualize it, we are going to;

1. get the path of `anomalies.pbtxt`from `example_validator`
2. parse `anomalies.pbtxt` into anomalies (protobuf)
3. visualize schema with [tfdv](https://www.tensorflow.org/tfx/data_validation/get_started)

In [ ]:
# 1. get the path of `schema.pbtxt`
def get_validation_directory(example_validator):
    output_dict = {key: value.get() for key, value in example_validator.outputs.get_all().items()}
    directory = types.get_split_uri(output_dict['output'], 'eval')
    return directory

validation_directory = get_validation_directory(example_validator)

In [ ]:
# 2. parse schema.pbtxt

from pathlib import Path
from google.protobuf.text_format import Parse
from google.protobuf.message import Message
from google.protobuf import text_format
from tensorflow_metadata.proto.v0 import anomalies_pb2

def parse_anomalies_proto_string(validation_directory):
    path = Path(validation_directory)
    for filepath in path.glob('*'):
        with open(str(filepath), 'r') as file: # since we are using python 3.5, not 3.6+
            anomalies_proto_string = file.read()
            anomalies = anomalies_pb2.Anomalies()
            text_format.Parse(anomalies_proto_string, anomalies)
            return anomalies

anomalies = parse_anomalies_proto_string(validation_directory)

In [ ]:
# 3. visualize anomalies with tfdv

import tensorflow_data_validation as tfdv
tfdv.display_anomalies(anomalies)

### Evaluate model

In [ ]:
def get_eval_dir(model_analyzer):
    artifact = model_analyzer.outputs.output.get()
    return types.get_single_uri(artifact)
    
eval_dir = get_eval_dir(model_analyzer)

In [ ]:
import tensorflow_model_analysis as tfma
result = tfma.load_eval_result(eval_dir)

In [ ]:
tfma.view.render_slicing_metrics(result, slicing_column='trip_start_hour')

In [ ]:
%config InlineBackend.figure_format = 'retina'
tfma.view.render_plot(result, tfma.slicer.SingleSliceSpec(features=[('trip_start_hour', 10)]))